# 1d FEM spaces: mass-, inter- and histopolation matrices

In [ ]:
import numpy as np

from struphy.feec.psydac_derham import Derham
from struphy.feec.mass import WeightedMassOperator

from psydac.ddm.cart import DomainDecomposition
from psydac.fem.tensor import TensorFemSpace
from psydac.core.bsplines import collocation_matrix, histopolation_matrix

In [ ]:
# instance of Derham
Nel = [12, 12, 12]  # Number of grid cells
p = [3, 4, 5]  # spline degrees
spl_kind = [True, True, True]  # Spline types (clamped vs. periodic)

derham = Derham(Nel, p, spl_kind)

## 1d spline spaces: attributes and point sets

In [ ]:
# 1d fem spaces

V0_fem = derham.Vh_fem["0"].spaces
V3_fem = derham.Vh_fem["3"].spaces

for l, (V0_1d, V3_1d) in enumerate(zip(V0_fem, V3_fem)):
    print(f"Direction {l + 1}")

    print("\nH1 p: ", V0_1d.degree)
    print("L2 p: ", V3_1d.degree)

    print("\nH1 knots: ", V0_1d.knots)
    print("L2 knots: ", V3_1d.knots)

    print("\nH1 basis: ", V0_1d.basis)
    print("L2 basis: ", V3_1d.basis)

    print("\nH1 nbasis: ", V0_1d.nbasis)
    print("L2 nbasis: ", V3_1d.nbasis)

    print("\nH1 breaks: ", V0_1d.breaks)
    print("L2 breaks: ", V3_1d.breaks)

    print("\nH1 greville: ", V0_1d.greville)
    print("L2 greville: ", V3_1d.greville)

    print("\nH1 ext_greville: ", V0_1d.ext_greville)
    print("L2 ext_greville: ", V3_1d.ext_greville)

    print("\n---------------------------------------")

## 1d mass matrices

$$
\mathbb M^0_{ij} := \int_{\hat \Omega} \Lambda^0_i\, \Lambda^0_j \sqrt g\,\textnormal d\eta
$$

$$
\mathbb M^1_{ij} := \int_{\hat \Omega} \Lambda^1_i\, \Lambda^1_j \frac{1}{\sqrt g}\,\textnormal d\eta
$$

$$
\mathbb M^{1\times 0}_{a^0, ij} := \int_{\hat \Omega} a^0 \Lambda^1_i \Lambda^0_j\,\textnormal d \eta
$$

For the moment, below all weights are equal to 1.

In [ ]:
# 1d mass matrices in H1 (no weight)
mass_H1_1d = []
for femspace_1d in V0_fem:
    domain_decompos_1d = DomainDecomposition([femspace_1d.ncells], [femspace_1d.periodic])
    femspace_1d_tensor = TensorFemSpace(domain_decompos_1d, femspace_1d)

    M = WeightedMassOperator(derham, femspace_1d_tensor, femspace_1d_tensor, nquads=[femspace_1d.degree])
    M.assemble(verbose=False)
    M.matrix.exchange_assembly_data()

    mass_H1_1d += [M.matrix.toarray()]

# 1d mass matrices in L2 (no weight)
mass_L2_1d = []
for femspace_1d in V3_fem:
    domain_decompos_1d = DomainDecomposition([femspace_1d.ncells], [femspace_1d.periodic])
    femspace_1d_tensor = TensorFemSpace(domain_decompos_1d, femspace_1d)

    M = WeightedMassOperator(derham, femspace_1d_tensor, femspace_1d_tensor, nquads=[femspace_1d.degree])
    M.assemble(verbose=False)
    M.matrix.exchange_assembly_data()

    mass_L2_1d += [M.matrix.toarray()]

# 1d mixed mass matrices: V0 -> V3
mass_mixed_1d = []
for V0_1d, V3_1d in zip(V0_fem, V3_fem):
    domain_decompos_1d = DomainDecomposition([V0_1d.ncells], [V0_1d.periodic])
    V0_femspace = TensorFemSpace(domain_decompos_1d, V0_1d)
    V3_femspace = TensorFemSpace(domain_decompos_1d, V3_1d)

    M = WeightedMassOperator(derham, V0_femspace, V3_femspace, nquads=[V0_1d.degree])
    M.assemble(verbose=False)
    M.matrix.exchange_assembly_data()

    mass_mixed_1d += [M.matrix.toarray()]

In [ ]:
print("Sorted eigenvalues of H1 mass matrices in 1d:")
for deg, M in zip(p, mass_H1_1d):
    print(f"\np={deg}:\n", np.sort(np.linalg.eigvals(M)))

print("\nSorted eigenvalues of L2 mass matrices in 1d:")
for deg, M in zip(p, mass_L2_1d):
    print(f"\np={deg}:\n", np.sort(np.linalg.eigvals(M)))

print("\nSorted eigenvalues (abs) of mixed mass matrices in 1d:")
for deg, M in zip(p, mass_mixed_1d):
    print(f"\np={deg}:\n", np.sort(np.abs(np.linalg.eigvals(M))))

In [ ]:
print("First row of circulant H1 mass matrices in 1d:")
for deg, M in zip(p, mass_H1_1d):
    print(f"\np={deg}:\n", M[0])

print("\nFirst row of circulant L2 mass matrices in 1d:")
for deg, M in zip(p, mass_L2_1d):
    print(f"\np={deg}:\n", M[0])

print("\nFirst row of circulant mixed mass matrices in 1d:")
for deg, M in zip(p, mass_mixed_1d):
    print(f"\np={deg}:\n", M[0])

## Standard collocation and histopolation matrices

$$
\mathcal C^0_{ij} := \sigma^0_i(\Lambda^0_j)\,, \qquad \mathcal H^1_{ij} := \sigma^1_i(\Lambda^1_j)\,.
$$

In [ ]:
# 1d Inter-/histopolation matrices

# Commuting projectors
P0 = derham.P["0"]
P3 = derham.P["3"]

# 1d collocation matrices
colloc_H1_1d = []
for mat in P0._imat.mats:
    colloc_H1_1d += [mat.toarray()]

# 1d histopolation matrices
histop_L2_1d = []
for mat in P3._imat.mats:
    histop_L2_1d += [mat.toarray()]

In [ ]:
print("Sorted eigenvalues of H1 collocation matrices in 1d:")
for deg, M in zip(p, colloc_H1_1d):
    print(f"\np={deg}:\n", np.sort(np.abs(np.linalg.eigvals(M))))

print("\nSorted eigenvalues of L2 histopolation matrices in 1d:")
for deg, M in zip(p, histop_L2_1d):
    print(f"\np={deg}:\n", np.sort(np.abs(np.linalg.eigvals(M))))

In [ ]:
print("First row of circulant H1 collocation matrices in 1d:")
for deg, M in zip(p, colloc_H1_1d):
    print(f"\np={deg}:\n", M[0])

print("\nFirst row of circulant L2 histopolation matrices in 1d:")
for deg, M in zip(p, histop_L2_1d):
    print(f"\np={deg}:\n", M[0])

## Opposite space collocation and histopolation matrices

$$
\mathcal C^1_{ij} := \sigma^0_i(\Lambda^1_j)\,, \qquad \mathcal H^0_{ij} := \sigma^1_i(\Lambda^0_j)\,.
$$

In [ ]:
# 1d Inter-/histopolation matrices of opposite spaces

# histopolation in H1
histop_H1_1d = []
for femspace_1d in V0_fem:
    hmat = histopolation_matrix(
        knots=femspace_1d.knots,
        degree=femspace_1d.degree,
        periodic=femspace_1d.periodic,
        normalization=femspace_1d.basis,
        xgrid=femspace_1d.greville,
    )

    histop_H1_1d += [hmat]

# interpolation in L2
colloc_L2_1d = []
for femspace_1d in V3_fem:
    imat = collocation_matrix(
        knots=femspace_1d.knots,
        degree=femspace_1d.degree,
        periodic=femspace_1d.periodic,
        normalization=femspace_1d.basis,
        xgrid=femspace_1d.ext_greville,
    )

    colloc_L2_1d += [imat]

In [ ]:
print("Sorted eigenvalues of H1 histopolation matrices in 1d:")
for deg, M in zip(p, histop_H1_1d):
    print(f"\np={deg}:\n", np.sort(np.abs(np.linalg.eigvals(M))))

print("\nSorted eigenvalues of L2 collocation matrices in 1d:")
for deg, M in zip(p, colloc_L2_1d):
    print(f"\np={deg}:\n", np.sort(np.abs(np.linalg.eigvals(M))))

In [ ]:
print("First row of circulant H1 histopolation matrices in 1d:")
for deg, M in zip(p, histop_H1_1d):
    print(f"\np={deg}:\n", M[0])

print("\nFirst row of circulant L2 collocation matrices in 1d:")
for deg, M in zip(p, colloc_L2_1d):
    print(f"\np={deg}:\n", M[0])